# 1. Get info box for one movie from Wikipedia

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
r=requests.get("https://en.wikipedia.org/wiki/Pathaan_(film)")
soup=bs(r.content)
contents = soup.prettify()
# print(contents)

In [3]:
info_box=soup.find(class_="infobox vevent")
info_rows=info_box.find_all("tr")

# for row in info_rows:
#     print(row.prettify())

In [4]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0"," ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0"," ")
movie_info={}

for index,row in enumerate(info_rows):
    if index==0 :
        movie_info['title']=row.find("th").get_text(" ", strip=True)
    elif index==1:
        continue
    else:
        try:
            content_key=row.find("th").get_text(" ", strip=True)
            content_value=get_content_value(row.find("td"))
            movie_info[content_key]=content_value
        except AttributeError:
            continue
        
        
    
# print(movie_info)

# 2. Get info box for all movies from Wikipedia

In [5]:
r=requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup=bs(r.content)
contents = soup.prettify()

# print(contents)

In [6]:
movies=soup.select(".wikitable.sortable i")
movies[0:10]
movies[0].a['title']

'Snow White and the Seven Dwarfs (1937 film)'

In [7]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0"," ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
        
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0"," ")

def clean(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()
        
def get_info_box(url):
    r=requests.get(url)
    soup=bs(r.content)
    
    info_box=soup.find(class_="infobox vevent")
    info_rows=info_box.find_all("tr")
    
    clean(soup)
    
    movie_info={}
    
    for index,row in enumerate(info_rows):
        if index==0 :
            movie_info['title']=row.find("th").get_text(" ", strip=True)
        else:
            try:
                header=row.find('th')
                if header:
                    content_key=row.find("th").get_text(" ", strip=True)
                    content_value=get_content_value(row.find("td"))
                    movie_info[content_key]=content_value
                
            except AttributeError:
                continue
    return movie_info


    

In [8]:
# testing clean sup
get_info_box('https://en.wikipedia.org/wiki/Davy_Crockett:_King_of_the_Wild_Frontier_(film)')

{'title': 'Davy Crockett: King of the Wild Frontier',
 'Directed by': 'Norman Foster',
 'Written by': 'Tom Blackburn',
 'Produced by': 'Bill Walsh',
 'Starring': ['Fess Parker', 'Buddy Ebsen'],
 'Cinematography': 'Charles P. Boyle',
 'Edited by': 'Chester W. Schaeffer',
 'Music by': ['Thomas W. Blackburn (lyrics)',
  'George Bruns',
  'Edward H. Plumb (orchestration)'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Film Distribution Co., Inc.',
 'Release date': 'May 25, 1955',
 'Running time': '93 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Box office': '$2,150,000 (US)'}

In [9]:
r=requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup=bs(r.content)
movies=soup.select(".wikitable.sortable i a")

base_path="https://en.wikipedia.org/"

movie_info_list=[]
for index,movie in enumerate(movies):
   
    try:
        relative_path=movie['href']
        
        full_path=base_path+relative_path
        title=movie['title']
        
        movie_info_list.append(get_info_box(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)


Elio
'NoneType' object has no attribute 'find_all'
61
'NoneType' object has no attribute 'find_all'
Big Thunder Mountain Railroad
'NoneType' object has no attribute 'find_all'
Keeper of the Lost Cities
'NoneType' object has no attribute 'find_all'
Muppet Man
'NoneType' object has no attribute 'find_all'
One Thousand and One Nights
'NoneType' object has no attribute 'find_all'
Space Mountain
'NoneType' object has no attribute 'find_all'
The Graveyard Book
'NoneType' object has no attribute 'find_all'
The Thief
'NoneType' object has no attribute 'find_all'
Tom Sawyer
'NoneType' object has no attribute 'find_all'
Tower of Terror
'NoneType' object has no attribute 'find_all'
Figment
'NoneType' object has no attribute 'find_all'


In [10]:
len(movie_info_list)

557

In [11]:
import json
def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data,f,ensure_ascii=False, indent=2)

In [12]:
import json

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [13]:
save_data("wiki_scrap_data.json", movie_info_list)

# 3. Clean the data

In [14]:
movie_info_list=load_data("wiki_scrap_data.json")

In [15]:
movie_info_list[-11]

{'title': 'SpaceCamp',
 'Directed by': 'Harry Winer',
 'Screenplay by': ['Clifford Green', 'Casey T. Mitchell'],
 'Story by': ['Patrick Bailey', 'Larry B. Williams'],
 'Produced by': ['Patrick Bailey', 'Walter Coblenz'],
 'Starring': ['Kate Capshaw',
  'Lea Thompson',
  'Kelly Preston',
  'Larry B. Scott',
  'Leaf Phoenix',
  'Tate Donovan',
  'Tom Skerritt'],
 'Cinematography': 'William A. Fraker',
 'Edited by': ['Tim Board', 'John W. Wheeler'],
 'Music by': 'John Williams',
 'Production company': 'ABC Motion Pictures',
 'Distributed by': '20th Century Fox',
 'Release date': ['June 6, 1986 (United States)'],
 'Running time': '107 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$18 million or $25 million',
 'Box office': '$9,697,739 (USA)'}

## 3.1 Running time clean

In [16]:
# [movie.get('Running time','NA') for movie in movie_info_list]

In [17]:
def min_to_intr(running_time):
    if running_time == "NA":
        return None
    
    if isinstance(running_time, list):
        entry=running_time[0]
        entry.replace("–"," ")
        return int(entry.split(" ")[0])
    else:
        return int(running_time.replace("–"," ").split(" ")[0])
        
    

for movie in movie_info_list:
    movie['Running time (int)']=min_to_intr(movie.get("Running time", "NA"))

In [18]:
# print([movie.get('Running time (int)','NA') for movie in movie_info_list])

## 3.2 Budget & BoxOffice clean

In [19]:
# print([movie.get("Budget","NA") for movie in movie_info_list])

In [20]:
import re

amounts= r"thousand|million|billion"
number= r"\d+(,\d{3})*\.*\d*"
word_re= rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"
value_re= rf"\${number}"

def word_to_value(word):
    value_dict={"thousand":1000, "million": 1000000, "billion":1000000000}
    return value_dict[word]

def parse_word_syntax(string):
    value_string=re.search(number, string).group()
    value=float(value_string.replace(",",""))
    word=re.search(amounts, string, flags=re.I).group().lower()
    word_value=word_to_value(word)
    return value*word_value

def parse_val_syntax(string):
    value_str=re.search(number,string).group()
    value=float(value_str.replace(",",""))
    return value

def money_conversion(money):
    if money=="NA":
        return None
    
    if isinstance(money, list):
        money=money[0]
    word_syntax=re.search(word_re, money, flags=re.I)
    val_syntax=re.search(value_re, money)
    if word_syntax:
        return parse_word_syntax(word_syntax.group())
    elif val_syntax:
        return parse_val_syntax(val_syntax.group())
    else:
        return None

In [21]:
for movie in movie_info_list:
    movie['Budget (float)']=money_conversion(movie.get("Budget", "NA"))
    movie['Box office (float)']=money_conversion(movie.get("Box office", "NA"))

In [22]:
movie_info_list[1]

{'title': 'Pinocchio',
 'Directed by': ['Ben Sharpsteen',
  'Hamilton Luske',
  'Bill Roberts',
  'Norman Ferguson',
  'Jack Kinney',
  'Wilfred Jackson',
  'T. Hee'],
 'Story by': ['Ted Sears',
  'Otto Englander',
  'Webb Smith',
  'William Cottrell',
  'Joseph Sabo',
  'Erdman Penner',
  'Aurelius Battaglia'],
 'Based on': ['The Adventures of Pinocchio', 'by', 'Carlo Collodi'],
 'Produced by': 'Walt Disney',
 'Starring': ['Cliff Edwards',
  'Dickie Jones',
  'Christian Rub',
  'Walter Catlett',
  'Charles Judels',
  'Evelyn Venable',
  'Frankie Darro'],
 'Music by': ['Leigh Harline', 'Paul J. Smith'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release dates': ['February 7, 1940 ( Center Theatre )',
  'February 23, 1940 (United States)'],
 'Running time': '88 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.6 million',
 'Box office': '$164 million',
 'Running time (int)': 88,
 'Budget (float)': 2600000.0,
 

## 3.3 Date clean

In [23]:
# print([movie.get("Release date", "NA") for movie in movie_info_list])

In [24]:
from datetime import datetime

dates=[movie.get("Release date", "NA") for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conv(date):
    if isinstance(date, list):
        date=date[0]
    if date=="NA":
        return None
    date_str=clean_date(date)
    
    formats=["%B %d, %Y", "%d %B %Y"]
    
    for form in formats:
        try:
            return datetime.strptime(date_str, form)
        except:
            pass
    return None
    

In [25]:
for movie in movie_info_list:
    if movie.get("Release date"):
            movie['Release date (datetime)']=date_conv(movie.get("Release date", "NA"))
    else:
            movie['Release date (datetime)']=date_conv(movie.get("Release dates", "NA"))

In [26]:
movie_info_list[100]

{'title': 'The Barefoot Executive',
 'Directed by': 'Robert Butler',
 'Screenplay by': 'Joseph L. McEveety',
 'Story by': ['Lila Garrett', 'Bernie Kahn', 'Stewart C. Billett'],
 'Produced by': 'Bill Anderson',
 'Starring': ['Kurt Russell',
  'Joe Flynn',
  'Wally Cox',
  'Heather North',
  'John Ritter',
  'Harry Morgan'],
 'Narrated by': 'Kurt Russell',
 'Cinematography': 'Charles F. Wheeler',
 'Edited by': 'Robert Stafford',
 'Music by': ['Robert F. Brunner', 'Franklyn Marks', 'Bruce Belland'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['March 17, 1971'],
 'Running time': '96 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Running time (int)': 96,
 'Budget (float)': None,
 'Box office (float)': None,
 'Release date (datetime)': datetime.datetime(1971, 3, 17, 0, 0)}

In [27]:
#save data using pickle

import pickle

def save_pickle(name,data):
    with open(name, "wb") as f:
        pickle.dump(data,f)

In [28]:
import pickle

def load_pickle(name):
    with open(name, "rb") as f:
        return pickle.load(f)

In [29]:
save_pickle("wiki_scrap_clean.pickle", movie_info_list)

In [30]:
a=load_pickle("wiki_scrap_clean.pickle")

In [31]:
a[1]

{'title': 'Pinocchio',
 'Directed by': ['Ben Sharpsteen',
  'Hamilton Luske',
  'Bill Roberts',
  'Norman Ferguson',
  'Jack Kinney',
  'Wilfred Jackson',
  'T. Hee'],
 'Story by': ['Ted Sears',
  'Otto Englander',
  'Webb Smith',
  'William Cottrell',
  'Joseph Sabo',
  'Erdman Penner',
  'Aurelius Battaglia'],
 'Based on': ['The Adventures of Pinocchio', 'by', 'Carlo Collodi'],
 'Produced by': 'Walt Disney',
 'Starring': ['Cliff Edwards',
  'Dickie Jones',
  'Christian Rub',
  'Walter Catlett',
  'Charles Judels',
  'Evelyn Venable',
  'Frankie Darro'],
 'Music by': ['Leigh Harline', 'Paul J. Smith'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release dates': ['February 7, 1940 ( Center Theatre )',
  'February 23, 1940 (United States)'],
 'Running time': '88 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.6 million',
 'Box office': '$164 million',
 'Running time (int)': 88,
 'Budget (float)': 2600000.0,
 

# 4. 3rd party ratings

In [32]:
movie_info_list=load_pickle("wiki_scrap_clean.pickle")

In [33]:
movie_info_list[-170]

{'title': 'The Crimson Wing: Mystery of the Flamingos',
 'Directed by': ['Matthew Aeberhard', 'Leander Ward'],
 'Written by': 'Melanie Finn',
 'Produced by': ['Matthew Aeberhard',
  'Leander Ward',
  'Paul Webster',
  '(executive producer)',
  'Stephen Garrett (executive producer)'],
 'Narrated by': ['Mariella Frostrup',
  '(',
  'UK',
  '/',
  'US',
  ')',
  'Zabou Breitman',
  '(',
  'France',
  ')'],
 'Music by': 'The Cinematic Orchestra and Claseria',
 'Production companies': ['Disneynature',
  'Natural Light Films',
  'Kudos Pictures'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release dates': ['26 October 2008 ( France )',
  '29 September 2009 ( United Kingdom )'],
 'Running time': '74 minutes',
 'Countries': ['United Kingdom', 'United States'],
 'Language': 'English',
 'Running time (int)': 74,
 'Budget (float)': None,
 'Box office (float)': None,
 'Release date (datetime)': datetime.datetime(2008, 10, 26, 0, 0)}

In [34]:
# http://www.omdbapi.com/?apikey=[yourkey]&

In [35]:
import requests
import urllib

def get_omdb(title):
    base_url="http://www.omdbapi.com/?"
    params={'t':title,"apikey": '9f97ffc'} #add your own api key
    params_encoded=urllib.parse.urlencode(params)
    
    full_url=base_url+params_encoded
#     print(full_url)
    
    return requests.get(full_url).json()

def get_rotten_score(omdb_info):
    ratings=omdb_info.get("Ratings",[])
    for rating in ratings:
        if rating["Source"]=='Rotten Tomatoes':
            return rating['Value']
    return None

get_omdb('pathaan')

{'Title': 'Pathaan',
 'Year': '2023',
 'Rated': 'Not Rated',
 'Released': '25 Jan 2023',
 'Runtime': '146 min',
 'Genre': 'Action, Adventure, Thriller',
 'Director': 'Siddharth Anand',
 'Writer': 'Shridhar Raghavan, Abbas Tyrewala, Siddharth Anand',
 'Actors': 'Shah Rukh Khan, Deepika Padukone, John Abraham',
 'Plot': 'An Indian agent races against a doomsday clock as a ruthless mercenary, with a bitter vendetta, mounts an apocalyptic attack against the country.',
 'Language': 'Hindi, English, Russian',
 'Country': 'India',
 'Awards': '3 nominations',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMThkZmNlMzQtZmU4Ny00YzhkLWE5YWYtOGJjNjRjODNjMzZkXkEyXkFqcGdeQXVyODE5NzE3OTE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.0/10'},
  {'Source': 'Metacritic', 'Value': '47/100'}],
 'Metascore': '47',
 'imdbRating': '6.0',
 'imdbVotes': '138,274',
 'imdbID': 'tt12844910',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': '$17,487,476',
 'Production': 'N/A',
 '

In [36]:
for movie in movie_info_list:
    title=movie['title']
    omdb_info=get_omdb(title)
    movie['imdb']=omdb_info.get('imdbRating', None)
    movie['rotten_tomatoes']=get_rotten_score(omdb_info)
    movie['metascore']=omdb_info.get("Metascore", None)
    

In [37]:
movie_info_list[4]

{'title': 'Dumbo',
 'Directed by': ['Ben Sharpsteen',
  'Norman Ferguson',
  'Wilfred Jackson',
  'Bill Roberts',
  'Jack Kinney',
  'Samuel Armstrong'],
 'Story by': ['Joe Grant', 'Dick Huemer'],
 'Based on': ['Dumbo, the Flying Elephant',
  'by',
  'Helen Aberson',
  'Harold Pearl'],
 'Produced by': 'Walt Disney',
 'Starring': ['Edward Brophy',
  'Verna Felton',
  'Cliff Edwards',
  'Herman Bing',
  'Sterling Holloway',
  'Margaret Wright',
  'Hall Johnson Choir'],
 'Narrated by': 'John McLeish',
 'Music by': ['Frank Churchill', 'Oliver Wallace'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release dates': ['October 23, 1941 (New York City)',
  'October 31, 1941 (U.S.)'],
 'Running time': '64 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$950,000',
 'Box office': '>$1.3 million (est. United States/Canada rentals, 1941)',
 'Running time (int)': 64,
 'Budget (float)': 950000.0,
 'Box office (float)': 1300000.

In [38]:
save_pickle("wiki_scrap_final.pickle", movie_info_list)

# 5. Save Dataset

In [39]:
movie_info_cp=[movie.copy() for movie in movie_info_list]

In [40]:
#converting datetime to string for storing in json
for movie in movie_info_cp:
    curr_date=movie['Release date (datetime)']
    if curr_date:
        movie['Release date (datetime)']=curr_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)']=None

In [41]:
movie_info_cp[269]

{'title': 'Snow Dogs',
 'Directed by': 'Brian Levant',
 'Written by': ['Jim Kouf',
  'Tommy Swerdlow',
  'Michael Goldberg',
  'Mark Gibson',
  'Philip Halprin'],
 'Based on': ['Winterdance', 'by', 'Gary Paulsen'],
 'Produced by': 'Jordan Kerner',
 'Starring': ['Cuba Gooding Jr.',
  'James Coburn',
  'Sisqó',
  'Nichelle Nichols',
  'Graham Greene'],
 'Cinematography': 'Thomas E. Ackerman',
 'Edited by': 'Roger Bondelli',
 'Music by': 'John Debney',
 'Production companies': ['Walt Disney Pictures',
  'The Kerner Entertainment Company'],
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['January 18, 2002'],
 'Running time': '95 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$33 million',
 'Box office': '$115 million',
 'Running time (int)': 95,
 'Budget (float)': 33000000.0,
 'Box office (float)': 115000000.0,
 'Release date (datetime)': 'January 18, 2002',
 'imdb': '5.2',
 'rotten_tomatoes': '25%',
 'metascore': '29'}

In [42]:
save_data("wiki_scrap_final.json", movie_info_cp)

In [43]:
#csv

import pandas as pd

df=pd.DataFrame(movie_info_list)


In [44]:
df.head()

,title,Directed by,Story by,Based on,Produced by,Starring,Music by,Production company,Distributed by,Release dates,...,Animated series,Television special(s),Play(s),Video game(s),Toy(s),Book(s),Radio program(s),Original music,Ice shows,Brand extension
0,Snow White and the Seven Dwarfs,"[David Hand, Perce Pearce, William Cottrell, L...","[Ted Sears, Richard Creedon, Otto Englander, D...","[Snow White, by the, Brothers Grimm]",Walt Disney,"[Adriana Caselotti, Roy Atwell, Pinto Colvig, ...","[Frank Churchill, Leigh Harline, Paul Smith]",Walt Disney Productions,RKO Radio Pictures,"[December 21, 1937 ( Carthay Circle Theatre ),...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pinocchio,"[Ben Sharpsteen, Hamilton Luske, Bill Roberts,...","[Ted Sears, Otto Englander, Webb Smith, Willia...","[The Adventures of Pinocchio, by, Carlo Collodi]",Walt Disney,"[Cliff Edwards, Dickie Jones, Christian Rub, W...","[Leigh Harline, Paul J. Smith]",Walt Disney Productions,RKO Radio Pictures,"[February 7, 1940 ( Center Theatre ), February...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fantasia,"[Samuel Armstrong, James Algar, Bill Roberts, ...","[Joe Grant, Dick Huemer]",NaN,"[Walt Disney, Ben Sharpsteen]","[Leopold Stokowski, Deems Taylor]",See program,Walt Disney Productions,RKO Radio Pictures,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,The Reluctant Dragon,"[Alfred Werker, (live action), Hamilton Luske,...",NaN,NaN,Walt Disney,"[Robert Benchley, Frances Gifford, Buddy Peppe...","[Frank Churchill, Larry Morey]",Walt Disney Productions,RKO Radio Pictures,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dumbo,"[Ben Sharpsteen, Norman Ferguson, Wilfred Jack...","[Joe Grant, Dick Huemer]","[Dumbo, the Flying Elephant, by, Helen Aberson...",Walt Disney,"[Edward Brophy, Verna Felton, Cliff Edwards, H...","[Frank Churchill, Oliver Wallace]",Walt Disney Productions,RKO Radio Pictures,"[October 23, 1941 (New York City), October 31,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df.to_csv("wiki_scrap_final.csv")

In [46]:
running_times=df.sort_values(["Running time (int)"], ascending=False)
running_times.head(10)

,title,Directed by,Story by,Based on,Produced by,Starring,Music by,Production company,Distributed by,Release dates,...,Animated series,Television special(s),Play(s),Video game(s),Toy(s),Book(s),Radio program(s),Original music,Ice shows,Brand extension
549,The Parent Trap,NaN,NaN,"[Lisa and Lottie, by, Erich Kästner]",NaN,NaN,NaN,NaN,The Walt Disney Company,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,Tinker Bell .mw-parser-output .nobold{font-wei...,"[Bradley Raymond (1, & 3), Klay Hall ( 2 ), Pe...",NaN,NaN,NaN,"[Mae Whitman, Lucy Liu, Raven-Symoné, ( More )]",Joel McNeely,DisneyToon Studios,"[Walt Disney Studios, Home Entertainment]","[1, : October 28, 2008, 2, :, 3, :, 4, :, 5, :...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,Pirates of the Caribbean: At World's End,Gore Verbinski,NaN,[Characters by Ted Elliott Terry Rossio Stuart...,Jerry Bruckheimer,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Hans Zimmer,NaN,Buena Vista Pictures Distribution,"[May 19, 2007 ( Disneyland Resort ), May 25, 2...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,The Happiest Millionaire,Norman Tokar,A. J. Carothers,"[My Philadelphia Father, by Cordelia Drexel Bi...","[Walt Disney, Bill Anderson]","[Fred MacMurray, Tommy Steele, Greer Garson, G...",Jack Elliott,Walt Disney Productions,Buena Vista Distribution,"[June 23, 1967, November 30, 1967]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
459,Jagga Jasoos,Anurag Basu,Anurag Basu,NaN,"[Siddharth Roy Kapur, Anurag Basu, Ranbir Kapoor]","[Ranbir Kapoor, Katrina Kaif, Saswata Chatterj...",Pritam,NaN,UTV Motion Pictures,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
452,Dangal,Nitesh Tiwari,"[Curation:, Nitesh Tiwari, Concept:, Divya V. ...",NaN,"[Aamir Khan, Kiran Rao, Siddharth Roy Kapur]","[Aamir Khan, Sakshi Tanwar, Fatima Sana Shaikh...",Pritam,NaN,UTV Motion Pictures,"[21 December 2016 (United States), 23 December...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,Hamilton,Thomas Kail,NaN,"[Alexander Hamilton, by, Ron Chernow]","[Thomas Kail, Lin-Manuel Miranda, Jeffrey Seller]","[Daveed Diggs, Renée Elise Goldsberry, Jonatha...",Lin-Manuel Miranda,NaN,Walt Disney Studios Motion Pictures,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504,The Beatles: Get Back,Peter Jackson,NaN,NaN,NaN,"[John Lennon, Paul McCartney, George Harrison,...",[The Beatles],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,ABCD 2,Remo D'Souza,Remo D'Souza,"[Suresh and Vernon, of, The Fictitious Crew]",Siddharth Roy Kapur,"[Prabhu Deva, Varun Dhawan, Shraddha Kapoor, S...",Sachin–Jigar,Walt Disney Pictures,UTV Motion Pictures,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,The Chronicles of Narnia: Prince Caspian,Andrew Adamson,NaN,"[Prince Caspian, by, C. S. Lewis]","[Mark Johnson, Andrew Adamson, Philip Steuer]","[Georgie Henley, Skandar Keynes, William Mosel...",Harry Gregson-Williams,NaN,Walt Disney Studios Motion Pictures,"[May 7, 2008 ( New York City ), May 16, 2008 (...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
